In [2]:
# Import required libraries
import pandas as pd
! pip install plotly
! pip install jupyter-dash
#! jupyter labextension link extensions/jupyterlab
#! jupyter nbextension install --sys-prefix --symlink --py jupyter_dash
#! jupyter nbextension enable --py jupyter_dash
#import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import io
import requests

# Read the airline data into pandas dataframe
url= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
s=requests.get(url).content
spacex_df=pd.read_csv(io.StringIO(s.decode('utf-8')))

#spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()



# Create a dash application
#app = dash.Dash(__name__)
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAF SLC-40', 'value': 'CCAFS SLC-40'}
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                    100: '100'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# # TASK 2:
# # Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
                                   Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(data, values='class',
        names='Launch Site',
        title='Total success lunches by site'+ entered_site)
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values = 'class count', names = 'class', title = 'Launch Success Counts on site')
        return fig

# # TASK 4:
# # Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='site-dropdown', component_property='value'),
                Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, payload):
    low, high = (payload[0],payload[1])
    dfa=spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    #filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(dfa, x='Payload Mass (kg)',y='class',
        title='Correlation betwwen Payload and Success for All sites', color = 'Booster Version Category')
        return fig
    else:
        dfb=dfa[dfa['Launch Site']==entered_site]
        #filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.scatter(dfb, x='Payload Mass (kg)',y='class',
        title='Correlation betwwen Payload and Success for the site'+entered_site, color = 'Booster Version Category')
        return fig
  

# Run the app
if __name__ == '__main__':
    app.run_server()
    #app.run_server(mode="inline", host="localhost") 

     |████████████████████████████████| 7.3 MB 42.6 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 23.9 MB 73.2 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 76.3 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 76.4 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3804 sha256=1c8e9a3adde8c2687e73b3fced43c4b5f72db1963abdf5f3b2d9ab7c05ecb86f
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/e4/f3/16724791571a955a46d54650510c98c04ab7d339626aee27cc
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=f4c0354dec8f187ec92c858ed2fa737ae5ed53719c3d06a3654137040e2cc7cc
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/73/d8/8d/92f612c03c895f19bcc56a6c54be7bb41aaa698012a5624f60
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl 

<ipython-input-1-7f37d6ce7352>:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-7f37d6ce7352>:11: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


OSError: Unable to communicate with the jupyter_dash notebook or JupyterLab 
extension required to infer Jupyter configuration.